In [29]:
import pandas as pd
import sys
import numpy as np
import matplotlib.pyplot as plt
from hyperparameters import hyperparameters
import matplotlib as mpl

from IPython.display import display
plt.style.use('bmh')
mpl.rcParams.update({
    "grid.linestyle" : "dashed",
    "axes.facecolor" : "white",
    "axes.spines.top" : False,
    "axes.spines.right" : False,
    "legend.frameon" : False,
    "figure.figsize" : (8, 5),
    "figure.dpi" : 300,
})
%matplotlib inline

# Suppress sklearn deprecated warnings
import warnings
def warn(*args, **kwargs): pass
warnings.warn = warn
np.set_printoptions(threshold=sys.maxsize)

np.random.seed(42)

In [30]:
# TODO make test for dataset with creatina column
# Dataset without thyroid = 18 features (including survive7y)
# Dataset with thyroid = 27 features (including survive7y)
# With columns that have missing values, 23 and 32
# Default 18
n_features = 27
extra_path = n_features != 27 and n_features != 18
dropped_na_key = "dropped_na/"
mean_key = "mean/"
key = mean_key
path = f"data/{n_features}features/{key if extra_path else '' }"
path_models = f"models/{n_features}features/{key if extra_path else '' }"
output_models = f"models_output/{n_features}features/{key if extra_path else '' }"
print(path_models)
print(path)
print(output_models)

models/27features/
data/27features/
models_output/27features/


In [31]:
# Read data
df_train = pd.read_csv(f"{path}train.csv", index_col=0)
df_valid = pd.read_csv(f"{path}valid.csv", index_col=0)
df_test = pd.read_csv(f"{path}test.csv", index_col=0)
print(len(df_train) + len(df_valid) + len(df_test))
print(len(df_train.columns))


train, valid, test = df_train.to_numpy(), df_valid.to_numpy(), df_test.to_numpy()

# y_**** contains the value of Survive7y as a list
# X_**** contains everything except for Survive7y as a list of list
X_train, y_train = train[:, :-1], train[:, -1]
X_valid, y_valid = valid[:, :-1], valid[:, -1]
X_test, y_test = test[:, :-1], test[:, -1]
feat_names = list(df_train.columns)
# Print how Survive7y are distribuited in each set
from collections import Counter
print(Counter(y_train))
print(Counter(y_valid))
print(Counter(y_test))

# All the numerical features that can be standardized
from utils import get_preprocess_std_num
preprocess_std = get_preprocess_std_num(feat_names)

# Preprocessed ready-to-use train and valid set
process_tmp = preprocess_std.fit(X_train)
X_train_std = process_tmp.transform(X_train)
X_valid_std = process_tmp.transform(X_valid)

#If you want to print the resulting df
# Note: You don't need to pass the _std to the train function. The function will call predict on the pipeline and transform the dataset accordingly to the transformer  
#df_scaled = pd.DataFrame(X_valid_std,columns = preprocess_std.get_feature_names_out())
#display(df_scaled)

6667
27
Counter({np.float64(1.0): 3494, np.float64(0.0): 505})
Counter({np.float64(1.0): 1165, np.float64(0.0): 169})
Counter({np.float64(1.0): 1165, np.float64(0.0): 169})


### Training


In [32]:
from functools import partial
from train import report, evaluate, train_and_evaluate
train_partial = partial(
    train_and_evaluate, 
    preprocess_std, 
    X_train=X_train,
    y_train=y_train,
    X_valid=X_valid,
    y_valid=y_valid,
    scoring="f1_macro", 
    iter=5000, 
    save=True,
    path_models = path_models,
    output_models = output_models
)

In [33]:
from sklearn.linear_model import LogisticRegression

hyperparams = hyperparameters["lr"] 
#Default is None (thus weight = 1). Balanced uses the formula n_samples / (n_classes * np.bincount(y))
model = LogisticRegression(class_weight="balanced")
train_partial(model=model, hyperparams=hyperparams, savename="lr")

/home/ileniag/buzi_ml4cad_0/venv/lib/python3.12/site-packages/sklearn/linear_model/_logistic.py:473: ConvergenceWarning: lbfgs failed to converge after 63 iteration(s) (status=1):
STOP: TOTAL NO. OF ITERATIONS REACHED LIMIT

Increase the number of iterations to improve the convergence (max_iter=63).
You might also want to scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
/home/ileniag/buzi_ml4cad_0/venv/lib/python3.12/site-packages/sklearn/linear_model/_logistic.py:473: ConvergenceWarning: lbfgs failed to converge after 63 iteration(s) (status=1):
STOP: TOTAL NO. OF ITERATIONS REACHED LIMIT

Increase the number of iterations to improve the convergence (max_iter=63).
You might also want to scale the data as shown in:
    https://scikit-learn.org/stable/modules

Testing on training set:
              precision    recall  f1-score   support

         0.0      0.399     0.576     0.471       505
         1.0      0.935     0.874     0.903      3494

    accuracy                          0.837      3999
   macro avg      0.667     0.725     0.687      3999
weighted avg      0.867     0.837     0.849      3999

auc macro 0.831
confusion matrix
[[ 291  214]
 [ 439 3055]]
Testing on validation set:
              precision    recall  f1-score   support

         0.0      0.414     0.686     0.517       169
         1.0      0.950     0.859     0.902      1165

    accuracy                          0.837      1334
   macro avg      0.682     0.773     0.709      1334
weighted avg      0.882     0.837     0.853      1334

auc macro 0.844
confusion matrix
[[ 116   53]
 [ 164 1001]]
Model rank: 1
Mean validation score: 0.694 (std: 0.002)
Parameters: {'model__C': 5, 'model__dual': True, 'model__max_iter': 294, 'model__penalty': 'l2', 'model__solver': 'lib

,steps,"[('preprocess', ...), ('model', ...)]"
,transform_input,None
,memory,None
,verbose,False
,transformers,"[('stand', ...)]"
,remainder,'passthrough'
,sparse_threshold,0.3
,n_jobs,None
,transformer_weights,None
,verbose,False
,verbose_feature_names_out,False


In [34]:
from sklearn.svm import SVC
hyperparams = hyperparameters["svc"] 

model = SVC(class_weight="balanced", probability=True)
train_partial(model=model, hyperparams=hyperparams, savename="svc")

/home/ileniag/buzi_ml4cad_0/venv/lib/python3.12/site-packages/sklearn/svm/_base.py:305: ConvergenceWarning: Solver terminated early (max_iter=800).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  warnings.warn(
/home/ileniag/buzi_ml4cad_0/venv/lib/python3.12/site-packages/sklearn/svm/_base.py:305: ConvergenceWarning: Solver terminated early (max_iter=800).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  warnings.warn(
/home/ileniag/buzi_ml4cad_0/venv/lib/python3.12/site-packages/sklearn/svm/_base.py:305: ConvergenceWarning: Solver terminated early (max_iter=800).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  warnings.warn(
/home/ileniag/buzi_ml4cad_0/venv/lib/python3.12/site-packages/sklearn/svm/_base.py:305: ConvergenceWarning: Solver terminated early (max_iter=1200).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  warnings.warn(
/home/ileniag/buzi_ml4cad_0/venv/lib/python3.12/sit

Testing on training set:
              precision    recall  f1-score   support

         0.0      0.234     0.434     0.304       505
         1.0      0.907     0.795     0.847      3494

    accuracy                          0.749      3999
   macro avg      0.570     0.614     0.576      3999
weighted avg      0.822     0.749     0.779      3999

auc macro 0.644
confusion matrix
[[ 219  286]
 [ 716 2778]]
Testing on validation set:
              precision    recall  f1-score   support

         0.0      0.204     0.408     0.272       169
         1.0      0.900     0.769     0.829      1165

    accuracy                          0.723      1334
   macro avg      0.552     0.589     0.551      1334
weighted avg      0.811     0.723     0.759      1334

auc macro 0.596
confusion matrix
[[ 69 100]
 [269 896]]
Model rank: 1
Mean validation score: 0.662 (std: 0.007)
Parameters: {'model__C': 119, 'model__coef0': np.float64(0.6971844289451361), 'model__degree': 151, 'model__gamma': 'scale

,steps,"[('preprocess', ...), ('model', ...)]"
,transform_input,None
,memory,None
,verbose,False
,transformers,"[('stand', ...)]"
,remainder,'passthrough'
,sparse_threshold,0.3
,n_jobs,None
,transformer_weights,None
,verbose,False
,verbose_feature_names_out,False


In [35]:
from sklearn.neighbors import KNeighborsClassifier

hyperparams = hyperparameters["knn"] 

model = KNeighborsClassifier()
train_partial(model=model, hyperparams=hyperparams, savename="knn")

Testing on training set:
              precision    recall  f1-score   support

         0.0      0.664     0.554     0.604       505
         1.0      0.937     0.959     0.948      3494

    accuracy                          0.908      3999
   macro avg      0.800     0.757     0.776      3999
weighted avg      0.903     0.908     0.905      3999

auc macro 0.937
confusion matrix
[[ 280  225]
 [ 142 3352]]
Testing on validation set:
              precision    recall  f1-score   support

         0.0      0.435     0.337     0.380       169
         1.0      0.907     0.936     0.921      1165

    accuracy                          0.861      1334
   macro avg      0.671     0.637     0.651      1334
weighted avg      0.847     0.861     0.853      1334

auc macro 0.698
confusion matrix
[[  57  112]
 [  74 1091]]
Model rank: 1
Mean validation score: 0.614 (std: 0.003)
Parameters: {'model__algorithm': 'ball_tree', 'model__leaf_size': 11, 'model__n_neighbors': 4, 'model__weights': 'unif

,steps,"[('preprocess', ...), ('model', ...)]"
,transform_input,None
,memory,None
,verbose,False
,transformers,"[('stand', ...)]"
,remainder,'passthrough'
,sparse_threshold,0.3
,n_jobs,None
,transformer_weights,None
,verbose,False
,verbose_feature_names_out,False


In [36]:
from sklearn.ensemble import RandomForestClassifier

hyperparams = hyperparameters["rf"] 

model = RandomForestClassifier()
train_partial(model=model, hyperparams=hyperparams, savename="rf")

Testing on training set:
              precision    recall  f1-score   support

         0.0      0.778     0.925     0.845       505
         1.0      0.989     0.962     0.975      3494

    accuracy                          0.957      3999
   macro avg      0.884     0.943     0.910      3999
weighted avg      0.962     0.957     0.959      3999

auc macro 0.989
confusion matrix
[[ 467   38]
 [ 133 3361]]
Testing on validation set:
              precision    recall  f1-score   support

         0.0      0.494     0.509     0.501       169
         1.0      0.928     0.924     0.926      1165

    accuracy                          0.872      1334
   macro avg      0.711     0.717     0.714      1334
weighted avg      0.873     0.872     0.873      1334

auc macro 0.833
confusion matrix
[[  86   83]
 [  88 1077]]
Model rank: 1
Mean validation score: 0.683 (std: 0.011)
Parameters: {'model__class_weight': 'balanced_subsample', 'model__criterion': 'gini', 'model__max_features': 'sqrt', '

,steps,"[('preprocess', ...), ('model', ...)]"
,transform_input,None
,memory,None
,verbose,False
,transformers,"[('stand', ...)]"
,remainder,'passthrough'
,sparse_threshold,0.3
,n_jobs,None
,transformer_weights,None
,verbose,False
,verbose_feature_names_out,False


In [37]:
from sklearn.ensemble import AdaBoostClassifier
from sklearn.tree import DecisionTreeClassifier

hyperparams = hyperparameters["adaboost"] 

model = AdaBoostClassifier()
train_partial(model=model, hyperparams=hyperparams, savename="adaboost")

Testing on training set:
              precision    recall  f1-score   support

         0.0      0.656     0.234     0.345       505
         1.0      0.899     0.982     0.939      3494

    accuracy                          0.888      3999
   macro avg      0.777     0.608     0.642      3999
weighted avg      0.868     0.888     0.864      3999

auc macro 0.847
confusion matrix
[[ 118  387]
 [  62 3432]]
Testing on validation set:
              precision    recall  f1-score   support

         0.0      0.667     0.260     0.374       169
         1.0      0.901     0.981     0.940      1165

    accuracy                          0.890      1334
   macro avg      0.784     0.621     0.657      1334
weighted avg      0.872     0.890     0.868      1334

auc macro 0.840
confusion matrix
[[  44  125]
 [  22 1143]]
Model rank: 1
Mean validation score: 0.655 (std: 0.008)
Parameters: {'model__learning_rate': np.float64(1.138925935534004), 'model__n_estimators': 81}

Model rank: 2
Mean val

,steps,"[('preprocess', ...), ('model', ...)]"
,transform_input,None
,memory,None
,verbose,False
,transformers,"[('stand', ...)]"
,remainder,'passthrough'
,sparse_threshold,0.3
,n_jobs,None
,transformer_weights,None
,verbose,False
,verbose_feature_names_out,False


In [38]:
from sklearn.neural_network import MLPClassifier
import random

hyperparams = hyperparameters["nn"] 

model = MLPClassifier()
train_partial(model=model, hyperparams=hyperparams, savename="nn")

Testing on training set:
              precision    recall  f1-score   support

         0.0      0.527     0.309     0.390       505
         1.0      0.906     0.960     0.932      3494

    accuracy                          0.878      3999
   macro avg      0.716     0.634     0.661      3999
weighted avg      0.858     0.878     0.864      3999

auc macro 0.828
confusion matrix
[[ 156  349]
 [ 140 3354]]
Testing on validation set:
              precision    recall  f1-score   support

         0.0      0.575     0.385     0.461       169
         1.0      0.915     0.959     0.936      1165

    accuracy                          0.886      1334
   macro avg      0.745     0.672     0.699      1334
weighted avg      0.872     0.886     0.876      1334

auc macro 0.845
confusion matrix
[[  65  104]
 [  48 1117]]
Model rank: 1
Mean validation score: 0.677 (std: 0.000)
Parameters: {'model__alpha': np.float64(0.5689420405848188), 'model__early_stopping': True, 'model__hidden_layer_sizes

,steps,"[('preprocess', ...), ('model', ...)]"
,transform_input,None
,memory,None
,verbose,False
,transformers,"[('stand', ...)]"
,remainder,'passthrough'
,sparse_threshold,0.3
,n_jobs,None
,transformer_weights,None
,verbose,False
,verbose_feature_names_out,False


In [39]:
from sklearn.ensemble import GradientBoostingClassifier

hyperparams = hyperparameters["gb"] 

model = GradientBoostingClassifier()
train_partial(model=model, hyperparams=hyperparams, savename="gb")

Testing on training set:
              precision    recall  f1-score   support

         0.0      0.757     0.321     0.451       505
         1.0      0.909     0.985     0.946      3494

    accuracy                          0.901      3999
   macro avg      0.833     0.653     0.698      3999
weighted avg      0.890     0.901     0.883      3999

auc macro 0.870
confusion matrix
[[ 162  343]
 [  52 3442]]
Testing on validation set:
              precision    recall  f1-score   support

         0.0      0.662     0.302     0.415       169
         1.0      0.906     0.978     0.941      1165

    accuracy                          0.892      1334
   macro avg      0.784     0.640     0.678      1334
weighted avg      0.875     0.892     0.874      1334

auc macro 0.842
confusion matrix
[[  51  118]
 [  26 1139]]
Model rank: 1
Mean validation score: 0.667 (std: 0.011)
Parameters: {'model__learning_rate': np.float64(0.1636229004837178), 'model__max_depth': 2, 'model__max_features': Non

,steps,"[('preprocess', ...), ('model', ...)]"
,transform_input,None
,memory,None
,verbose,False
,transformers,"[('stand', ...)]"
,remainder,'passthrough'
,sparse_threshold,0.3
,n_jobs,None
,transformer_weights,None
,verbose,False
,verbose_feature_names_out,False


In [40]:
#Don't run this in jupyter within vscode, run this with notebooks within browsers.
import os
#os.environ['KMP_DUPLICATE_LIB_OK']='True'

import xgboost as xgb

hyperparams = hyperparameters["xgb"] 

model = xgb.XGBClassifier(n_jobs=1)
train_partial(model=model, hyperparams=hyperparams, savename="xgb")

/home/ileniag/buzi_ml4cad_0/venv/lib/python3.12/site-packages/xgboost/training.py:183: UserWarning: [20:34:24] WARNING: /workspace/src/learner.cc:738: 
Parameters: { "gamma", "max_depth", "subsample" } are not used.

  bst.update(dtrain, iteration=i, fobj=obj)
/home/ileniag/buzi_ml4cad_0/venv/lib/python3.12/site-packages/xgboost/training.py:183: UserWarning: [20:34:24] WARNING: /workspace/src/learner.cc:738: 
Parameters: { "gamma", "max_depth", "subsample" } are not used.

  bst.update(dtrain, iteration=i, fobj=obj)
/home/ileniag/buzi_ml4cad_0/venv/lib/python3.12/site-packages/xgboost/training.py:183: UserWarning: [20:34:24] WARNING: /workspace/src/learner.cc:738: 
Parameters: { "gamma", "max_depth", "subsample" } are not used.

  bst.update(dtrain, iteration=i, fobj=obj)
/home/ileniag/buzi_ml4cad_0/venv/lib/python3.12/site-packages/xgboost/training.py:183: UserWarning: [20:34:24] WARNING: /workspace/src/learner.cc:738: 
Parameters: { "gamma", "max_depth", "subsample" } are not used.



Testing on training set:
              precision    recall  f1-score   support

         0.0      0.519     0.556     0.537       505
         1.0      0.935     0.926     0.930      3494

    accuracy                          0.879      3999
   macro avg      0.727     0.741     0.734      3999
weighted avg      0.883     0.879     0.881      3999

auc macro 0.875
confusion matrix
[[ 281  224]
 [ 260 3234]]
Testing on validation set:
              precision    recall  f1-score   support

         0.0      0.456     0.527     0.489       169
         1.0      0.930     0.909     0.919      1165

    accuracy                          0.861      1334
   macro avg      0.693     0.718     0.704      1334
weighted avg      0.870     0.861     0.865      1334

auc macro 0.832
confusion matrix
[[  89   80]
 [ 106 1059]]
Model rank: 1
Mean validation score: 0.699 (std: 0.003)
Parameters: {'model__alpha': np.float64(0.4861950368908073), 'model__booster': 'dart', 'model__eta': np.float64(0.4895

,steps,"[('preprocess', ...), ('model', ...)]"
,transform_input,None
,memory,None
,verbose,False
,transformers,"[('stand', ...)]"
,remainder,'passthrough'
,sparse_threshold,0.3
,n_jobs,None
,transformer_weights,None
,verbose,False
,verbose_feature_names_out,False


In [41]:

mean_path = f"data/27features/"
df_train = pd.read_csv(f"{mean_path}train.csv")
df_valid = pd.read_csv(f"{mean_path}valid.csv")
df_test = pd.read_csv(f"{mean_path}test.csv")
sum_valid = 0
sum_test = 0
print(1 in df_valid.iloc[:,0].to_numpy())
for val in df_train.iloc[:,0].to_numpy():
    if val in df_valid.iloc[:,0].to_numpy():
        print(val)
        sum_valid +=1
    if val in df_test.iloc[:,0].to_numpy():
        sum_test +=1
print("#######################")
print(sum_valid)
print(sum_test)

True
#######################
0
0


In [42]:
from auto_export_notebook import export_current_notebook


html_path = export_current_notebook(
    globals(),
    wait_for_disk_save=True,   # wait for Auto Save
    wait_timeout_sec=8.0
)
print("Exported to:", html_path)


<IPython.core.display.Javascript object>

Exported to: /home/ileniag/buzi_ml4cad_0/exported_notebooks/2_classifiers_27features_20250929_203957.html
